In [15]:
#TODO
#1. Add the 64*64 outputs to one image.
#2. For every pixel, convert it into the most probable one.Maybe depend on neighbours??
# Could be multiple overlaps, find a way to handle that.
#3. Convert every pixel to correct polygon json format

import numpy as np
import cv2
import os

In [7]:
def post_process_optimized(outputs, size=256, gamma=0.5):
    empty_img = np.zeros((1024, 1024, 5))
    processed_img = np.zeros((1024, 1024), dtype=int)
    
    for x, y, probs in outputs:
        empty_img[x:x+size, y:y+size] += np.array(probs)
    
    padded = np.pad(empty_img, ((1,1), (1,1), (0,0)))
    neighbors = gamma * (padded[:-2, 1:-1] + padded[2:, 1:-1] + 
                        padded[1:-1, :-2] + padded[1:-1, 2:])
    
    processed_img = np.argmax(empty_img + neighbors, axis=2)
    return processed_img

def test_post_process_optimized(size=256):
    positions = np.mgrid[0:1024:size, 0:1024:size].reshape(2, -1).T
    
    probs = np.random.dirichlet(np.ones(5)*2, size=positions.shape[0])
    probs[:,0] = np.clip(probs[:,0], 0.3, 1.0)
    probs = probs / probs.sum(axis=1, keepdims=True)
    
    outputs = [(x, y, p.tolist()) for (x, y), p in zip(positions, probs)]
    return post_process_optimized(outputs)


In [5]:
import torch

def post_process_torch(outputs, gamma=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    empty_img = torch.zeros((1024, 1024, 5), device=device)
    
    # Convert outputs to tensor once
    coords = torch.tensor([[x,y] for x,y,_ in outputs], device=device)
    probs = torch.tensor([p for _,_,p in outputs], device=device)
    
    # Batch assignment
    for coord, prob in zip(coords, probs):
        empty_img[coord[0]:coord[0]+64, coord[1]:coord[1]+64] += prob
    
    padded = torch.nn.functional.pad(empty_img, (0,0,1,1,1,1))
    neighbors = gamma * (padded[:-2, 1:-1] + padded[2:, 1:-1] + 
                        padded[1:-1, :-2] + padded[1:-1, 2:])
    
    return torch.argmax(empty_img + neighbors, dim=2)  # Keep on GPU if needed

In [6]:
# Example: A random 1024x1024 tensor with values 0-4
def converter(tensor):
# Dictionary to store polygons for each unique value
    polygons = {}

    for val in range(5):  # Iterate over unique values (0-4)
        mask = (tensor == val).astype(np.uint8)  # Create binary mask for the value

        # Find contours (polygons) of connected components
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Convert contours to a list of polygons
        polygons[val] = [c.reshape(-1, 2).tolist() for c in contours]
    return polygons
test=converter(test_post_process_optimized())
for i in range(5):
    print(test[i])
    print(len(test[i]))

[[[512, 959]], [[831, 896], [832, 896]], [[768, 896], [768, 959], [767, 960], [704, 960], [704, 1023], [767, 1023], [767, 960], [768, 959]], [[703, 896]], [[383, 896], [383, 959], [383, 897], [384, 896]], [[576, 895], [575, 896], [575, 959], [576, 960], [639, 960], [640, 959], [639, 958], [639, 897], [640, 896], [639, 895]], [[831, 831]], [[831, 768]], [[704, 768], [766, 768], [767, 769], [767, 831], [768, 831], [767, 830], [767, 769], [768, 768]], [[833, 704], [894, 704], [895, 705], [895, 766], [894, 767], [833, 767], [895, 767], [896, 768], [896, 832], [958, 832], [959, 833], [959, 894], [958, 895], [896, 895], [895, 896], [896, 895], [959, 895], [960, 896], [960, 959], [959, 960], [896, 960], [895, 959], [831, 959], [832, 960], [832, 1023], [1023, 1023], [1023, 832], [961, 832], [959, 830], [959, 768], [960, 767], [1023, 767], [1023, 704]], [[0, 703]], [[705, 640], [704, 641], [704, 704], [766, 704], [767, 703], [767, 640]], [[639, 319]], [[255, 319]], [[703, 256], [704, 256]], [[6

In [19]:
test_post_process_optimized()

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 4, 4, 4],
       [1, 1, 1, ..., 4, 4, 4],
       [1, 1, 1, ..., 4, 4, 4]])

In [10]:
import satlaspretrain_models
weights_manager = satlaspretrain_models.Weights()
device = torch.device('cpu')
model = weights_manager.get_pretrained_model("Sentinel2_SwinT_SI_MS", fpn=True, 
                                             head=satlaspretrain_models.Head.SEGMENT, 
                                                num_categories=5, device='cpu')
model = model.to(device)

In [13]:
def load_model(save_path):
    # 1. Initialize model architecture
    
    # 2. Load the saved weights
    weights_path = save_path + '_model_weights.pth'
    model.load_state_dict(torch.load(weights_path))
    
    # 3. Set to evaluation mode
    model.eval()
    
    return model

In [16]:
save_path = os.path.split(os.getcwd())[0] + '/weights/'
loaded_model = load_model(save_path)